In [1]:
import numpy as np

human_features = np.load('data/human_features.npy')
ai_features = np.load('data/ai_features.npy')

human_labels = np.zeros(len(human_features))
ai_labels = np.ones(len(ai_features))


X = np.concatenate([human_features, ai_features], axis=0)
y = np.concatenate([human_labels, ai_labels], axis=0)

In [2]:
len(ai_features), len(human_labels)

(2648, 6847)

In [3]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
                                X, y, test_size=0.2, stratify=y, random_state=42
)

In [4]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

rfc = RandomForestClassifier(n_estimators=100, max_depth=10,
                             class_weight={0: 1, 1: 3.5}, min_samples_leaf=3, random_state=1)
rfc.fit(X_train, y_train)

RandomForestClassifier(class_weight={0: 1, 1: 3.5}, max_depth=10,
                       min_samples_leaf=3, random_state=1)

In [45]:
import mlflow.sklearn
from datetime import datetime

features_names = ['sentense_avg_quantity', 'sentense_avg', 'word_avg','count_foreign', 
                'ADJ', 'ADP', 'ADV', 'AUX', 'CCONJ', 'DET', 'INTJ', 'NOUN', 'NUM', 
                'PART', 'PRON', 'PROPN', 'PUNCT', 'SCONJ', 'SYM', 'VERB', 'X',
                'words_divercity', 'text_ease', 'ext_punctuation_per_word', 
                'avg_dependency_path', 'unique_rate']


def check_results(trained_model, test = X_test,
                  brief:bool = True, save:bool = False,
                  saving_name:str = 'model_with_no_name'):
    y_pred = trained_model.predict(test)
    score = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)  
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)

    time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

    print(time, end='\n\n')
    print(score, precision, recall, f1, '\n', sep='\n')
    
    if brief:
        return 0
        
    feature_importances = [float(i) for i in trained_model.feature_importances_]
    features_and_importances = {}
    for i in range(len(features_names)):
        features_and_importances[features_names[i]] = feature_importances[i]
    
    print("Feature Importances:")
    for key, value in features_and_importances.items():
        print(f"{key}: {value:.4f}")
    if save:
        mlflow.set_experiment(f'{time}  experiment')
        with mlflow.start_run() as run:
            mlflow.sklearn.log_model(trained_model, saving_name)
            run_id = run.info.run_id
            print(f"RUN_ID для загрузки: {run_id}")

#check_results(rfc, brief=True)
check_results(rfc, brief=False, save=True, saving_name = 'rfc_1')

#the first results from random forest with manual parameters setting is quite good


2025/11/27 22:26:28 INFO mlflow.tracking.fluent: Experiment with name '2025-11-27 22:26:28  experiment' does not exist. Creating a new experiment.
2025/11/27 22:26:28 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


2025-11-27 22:26:28

0.9605055292259084
0.9366602687140115
0.9207547169811321
0.928639391056137


Feature Importances:
sentense_avg_quantity: 0.1353
sentense_avg: 0.0328
word_avg: 0.0479
count_foreign: 0.0182
ADJ: 0.0145
ADP: 0.0104
ADV: 0.0914
AUX: 0.0047
CCONJ: 0.0006
DET: 0.1054
INTJ: 0.1158
NOUN: 0.0096
NUM: 0.0075
PART: 0.0117
PRON: 0.0066
PROPN: 0.0046
PUNCT: 0.0000
SCONJ: 0.0290
SYM: 0.0001
VERB: 0.0544
X: 0.0370
words_divercity: 0.0463
text_ease: 0.0255
ext_punctuation_per_word: 0.0742
avg_dependency_path: 0.0035
unique_rate: 0.0937


2025/11/27 22:26:30 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


RUN_ID для загрузки: 64ace1a54ddf44d3999096807cc0d949


In [6]:
from sklearn.svm import SVC
from sklearn.naive_bayes import ComplementNB
from sklearn.linear_model import LogisticRegressionCV
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

svc = SVC(kernel='rbf', degree=3, C=1.0, class_weight={0: 1, 1: 3})
nb = ComplementNB(alpha=0.5, fit_prior=False, norm=False)
lr = LogisticRegressionCV(penalty='l2', class_weight={0: 1, 1: 3.5})

for model in [svc, nb, lr]:
    model.fit(X_train_scaled, y_train)
    check_results(model, X_test_scaled)

#SVM is worse than forest with f1 = 0.88 but it may have potential
#Naive bayes works poorly
#logistic regression is not bad but even svm is 5% better

2025-11-27 20:31:22

0.9394418114797262
0.8711985688729875
0.9188679245283019
0.8943985307621671


2025-11-27 20:31:22

0.6966824644549763
0.47248803827751196
0.7452830188679245
0.5783308931185944




/Users/iaroslav/Desktop/human_ai/venv/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/iaroslav/Desktop/human_ai/venv/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/l

2025-11-27 20:31:25

0.8957345971563981
0.7610062893081762
0.9132075471698113
0.8301886792452831




/Users/iaroslav/Desktop/human_ai/venv/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [ ]:
"""
Tuning parameters for svm and random forest only
"""
from skopt import BayesSearchCV

param_space_forest = {
    'n_estimators': (80, 200),
    'max_depth': (5, 25),
    'max_features': ['sqrt', 'log2', 0.5, 0.7],
    'class_weight': ['balanced', 'balanced_subsample'],
    'min_samples_leaf': (2, 4),
    'min_samples_split':(2, 4)
}

bayes_search_forest = BayesSearchCV(
    RandomForestClassifier(random_state=42),
    param_space_forest,
    n_iter=50,
    cv=4,
    scoring='f1',  
    random_state=42
)
bayes_search_forest.fit(X_train, y_train)
print('RandomForestClassifier')
print(bayes_search_forest.best_params_)
print(bayes_search_forest.best_score_, end='\n\n')


RandomForestClassifier
OrderedDict([('class_weight', 'balanced'), ('max_depth', 16), ('max_features', 'log2'), ('min_samples_leaf', 2), ('min_samples_split', 2), ('n_estimators', 84)])
0.9379721375642747



In [40]:
param_space_svm = {
    'kernel': ['rbf', 'poly', 'sigmoid'],
    'degree': (2, 5),
    'gamma': ('scale', 'auto'),
    'C': (0.1, 5),
    'class_weight':['balanced'],
}#break_ties = True, 

bayes_search_svm = BayesSearchCV(
    SVC(),
    param_space_svm,
    n_iter=10,  #searching runs too long with svm
    cv=5,
    scoring='f1',  
    random_state=42
)


bayes_search_svm.fit(X_train, y_train)
print('SupportVectorClassifier')
print(bayes_search_svm.best_params_)
print(bayes_search_svm.best_score_)

SupportVectorClassifier
OrderedDict([('C', 3.1236888566613503), ('class_weight', 'balanced'), ('degree', 3), ('gamma', 'auto'), ('kernel', 'poly')])
0.9320183736952758


In [ ]:
rfc_best = RandomForestClassifier(n_estimators=132, max_depth=16, max_features='log2',
                                  min_samples_leaf=3, min_samples_split=2,
                                  class_weight={0: 1, 1: 3.5}, random_state=21)
rfc_best.fit(X_train, y_train)
check_results(rfc_best, brief=False, save=True, saving_name='rfc_best')

#corrected n_estimators ->132, min_samples_leaf ->3 with new class_weigths 
#f1 score improved about 0.4% (not really much😔)           

2025/11/27 22:37:33 INFO mlflow.tracking.fluent: Experiment with name '2025-11-27 22:37:33  experiment' does not exist. Creating a new experiment.
2025/11/27 22:37:33 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


2025-11-27 22:37:33

0.9626119010005266
0.9526627218934911
0.9113207547169812
0.9315332690453231


Feature Importances:
sentense_avg_quantity: 0.1269
sentense_avg: 0.0320
word_avg: 0.0534
count_foreign: 0.0193
ADJ: 0.0197
ADP: 0.0112
ADV: 0.0727
AUX: 0.0055
CCONJ: 0.0010
DET: 0.0952
INTJ: 0.1126
NOUN: 0.0122
NUM: 0.0106
PART: 0.0151
PRON: 0.0097
PROPN: 0.0065
PUNCT: 0.0000
SCONJ: 0.0297
SYM: 0.0003
VERB: 0.0554
X: 0.0373
words_divercity: 0.0497
text_ease: 0.0351
ext_punctuation_per_word: 0.0637
avg_dependency_path: 0.0052
unique_rate: 0.0957


2025/11/27 22:37:35 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


RUN_ID для загрузки: 848088676de540cabae2b392671bc6e1


In [43]:
rfc_best = RandomForestClassifier(n_estimators=84, max_depth=16, max_features='log2',
                                  min_samples_leaf=2, min_samples_split=2,
                                  class_weight='balanced')
rfc_best.fit(X_train, y_train)
check_results(rfc_best, brief=False, save=True, saving_name='rfc_best_really')


2025/11/27 22:22:55 INFO mlflow.tracking.fluent: Experiment with name '2025-11-27 22:22:55  experiment' does not exist. Creating a new experiment.
2025/11/27 22:22:55 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


2025-11-27 22:22:55

0.9599789362822538
0.963265306122449
0.8905660377358491
0.9254901960784314


Feature Importances:
sentense_avg_quantity: 0.1336
sentense_avg: 0.0313
word_avg: 0.0511
count_foreign: 0.0226
ADJ: 0.0203
ADP: 0.0115
ADV: 0.0780
AUX: 0.0063
CCONJ: 0.0011
DET: 0.0906
INTJ: 0.1049
NOUN: 0.0118
NUM: 0.0111
PART: 0.0150
PRON: 0.0080
PROPN: 0.0068
PUNCT: 0.0000
SCONJ: 0.0331
SYM: 0.0005
VERB: 0.0574
X: 0.0405
words_divercity: 0.0486
text_ease: 0.0369
ext_punctuation_per_word: 0.0643
avg_dependency_path: 0.0047
unique_rate: 0.0860


2025/11/27 22:22:57 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
